# Shopping Cart Abandonment Prediction
## LSTM

LSTM model based on "Predicting purchasing intent: Automatic Feature Learning using
Recurrent Neural Networks" (Sheil, et.al.).

In [7]:
!wget https://github.com/mease/csci5502-cart-abandonment/raw/main/data/cart_abandon_data.zip
!mkdir data
!unzip cart_abandon_data.zip -d data

--2021-11-03 01:47:13--  https://github.com/mease/csci5502-cart-abandonment/raw/main/data/cart_abandon_data.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/mease/csci5502-cart-abandonment/main/data/cart_abandon_data.zip [following]
--2021-11-03 01:47:14--  https://media.githubusercontent.com/media/mease/csci5502-cart-abandonment/main/data/cart_abandon_data.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160490795 (153M) [application/zip]
Saving to: ‘cart_abandon_data.zip’

cart_abandon_data.z 100%[===================>] 153.06M   171MB/s    in 0.9s    

2021-11-03 01:47:20

In [8]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, LSTM, Dense

import numpy as np
import pickle

In [9]:
# Click sequence for all data points
sequence_length = 20

Load data:

In [10]:
with open('data/X_train.npy', 'rb') as f:
    X_train = np.load(f)
with open('data/y_train.npy', 'rb') as f:
    y_train = np.load(f)
with open('data/X_test.npy', 'rb') as f:
    X_test = np.load(f)
with open('data/y_test.npy', 'rb') as f:
    y_test = np.load(f)

In [11]:
for i in range(10):
    print(X_train[i], y_train[i])

[[ 3975  3563    32]
 [ 3975  2622    32]
 [ 3975 23268    32]
 [ 3975 21636     2]
 [ 3975  8388    27]
 [ 3975 43686    27]
 [ 3975 45848    27]
 [ 3975  8396    27]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]] 0
[[ 3465 45939    33]
 [ 3465 45939    33]
 [ 3465 45938    33]
 [ 3465 45938    33]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]] 0
[[ 1471 37295     1]
 [ 1471 37269     1]
 [ 1471 37269     1]
 [ 1471 37275     1]
 [ 1471 37275     1]
 [ 1471 37310     1]
 [ 1471 37310     1]
 [    0

Determine vocabulary size for feature embeddings.

In [12]:
time_vocab_size = len(np.unique(np.concatenate((np.unique(X_train[:,:,0]),np.unique(X_test[:,:,0])))))
item_vocab_size = len(np.unique(np.concatenate((np.unique(X_train[:,:,1]),np.unique(X_test[:,:,1])))))
cat_vocab_size = len(np.unique(np.concatenate((np.unique(X_train[:,:,2]),np.unique(X_test[:,:,2])))))
print(f"Time vocab: {time_vocab_size}, Item vocab: {item_vocab_size}, Category vocab: {cat_vocab_size}")

Time vocab: 4369, Item vocab: 51287, Category vocab: 330


Define the model:

In [13]:
# Timestamp input and embedding
time_inp = Input(shape=(sequence_length,), name='time_input')
time_emb = Embedding(output_dim=10,
                     input_dim=time_vocab_size,
                     input_length=sequence_length,
                     mask_zero=True,
                     name='time_embedding')(time_inp)

# Item ID input and embedding
item_inp = Input(shape=(sequence_length,), name='item_input')
item_emb = Embedding(output_dim=100,
                     input_dim=item_vocab_size,
                     input_length=sequence_length,
                     mask_zero=True,
                     name='item_embedding')(item_inp)

# Category input and embedding
cat_inp = Input(shape=(sequence_length,), name='category_input')
cat_emb = Embedding(output_dim=10,
                    input_dim=cat_vocab_size,
                    input_length=sequence_length,
                    mask_zero=True,
                    name='category_embedding')(cat_inp)

# Concatenate embeddings
concat_emb = Concatenate(axis=2, name='concat_embeddings')([time_emb, item_emb, cat_emb])

# LSTM
#lstm = LSTM(256, dropout=0.2, recurrent_dropout=0.2)(concat_emb)
lstm1 = LSTM(256, return_sequences=True, name='lstm1')(concat_emb)
lstm2 = LSTM(256, return_sequences=True, name='lstm2')(lstm1)
lstm3 = LSTM(256, name='lstm3')(lstm2)

# Output layer
out = Dense(1, activation='sigmoid', name='output')(lstm3)

model = Model(inputs=[time_inp, item_inp, cat_inp], outputs=[out])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[tf.keras.metrics.AUC()])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
time_input (InputLayer)         [(None, 20)]         0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 20)]         0                                            
__________________________________________________________________________________________________
category_input (InputLayer)     [(None, 20)]         0                                            
__________________________________________________________________________________________________
time_embedding (Embedding)      (None, 20, 10)       43690       time_input[0][0]                 
______________________________________________________________________________________________

Train model:

In [14]:
model.fit([X_train[:,:,0], X_train[:,:,1], X_train[:,:,2]], y_train,
          validation_data=([X_test[:,:,0], X_test[:,:,1], X_test[:,:,2]], y_test),
          epochs=4,
          batch_size=256)

Epoch 1/4
28690/28690 [==============================] - 1418s 49ms/step - loss: 0.1742 - auc: 0.8022 - val_loss: 0.1690 - val_auc: 0.8229
Epoch 2/4
28690/28690 [==============================] - 1397s 49ms/step - loss: 0.1655 - auc: 0.8338 - val_loss: 0.1672 - val_auc: 0.8280
Epoch 3/4
28690/28690 [==============================] - 1348s 47ms/step - loss: 0.1621 - auc: 0.8440 - val_loss: 0.1670 - val_auc: 0.8293
Epoch 4/4
28690/28690 [==============================] - 1369s 48ms/step - loss: 0.1591 - auc: 0.8519 - val_loss: 0.1678 - val_auc: 0.8273
